In [73]:
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch.nn as nn
from tqdm.notebook import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
dataset = datasets.fetch_california_housing()

In [3]:
data, target = dataset.get("data"), dataset.get("target")

In [4]:
scaler = StandardScaler()
data = scaler.fit_transform(data)


In [5]:
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

In [6]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, target):
        super(CustomDataset).__init__()
        self.data = data
        self.target = target
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        data = self.data[idx]
        target = self.target[idx]
        return {
            "data": torch.tensor(data, dtype=torch.float32).to(device),
            "target": torch.tensor(target, dtype = torch.float32).to(device)
        }

In [7]:
train_dataset = CustomDataset(x_train, y_train)
test_dataset = CustomDataset(x_test, y_test)

In [8]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size = 4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size = 4
)

## Model Builder

In [9]:
model = lambda x, w, b: (torch.matmul(x, w) + b)

Calculating Linear Regression 

- Compute the Output 
$$Y_{hat} = W^TX + b$$

- Calculate Loss MSE
$$MSE = \frac{1}{n}\Sigma_0^n(y - y_{hat}) ^2$$

In [10]:
W = torch.randn(8, 1, requires_grad=True, device=device)
b = torch.randn(1, requires_grad=True, device=device)

-1.955566644668579

In [14]:
learning_rate = 5e-3
best_loss, best_W, best_b = None, None, None
for epoch in range(10):
    epoch_loss = []
    for data in train_loader:
        x, y = data["data"], data["target"]
        output = model(x, W, b)
        
        loss = torch.mean((y.view(-1) - output.view(-1)) ** 2)
        loss.backward() # TO calculate the gradients
        epoch_loss.append(loss.item())
        
#         print(f"{epoch=}: {loss=}")
        with torch.no_grad(): # Its a context manager, it will disable all the previous requirements like required_grads
            W = W - (learning_rate * W.grad)
            b = b - (learning_rate * b.grad)
        
        # Refresh the context with requiring the grads
        W.requires_grad_(True)
        b.requires_grad_(True)
    mean_epoch_loss = torch.mean(torch.from_numpy(np.array(epoch_loss)))
    if best_loss == None or best_loss > mean_epoch_loss.item():
        best_loss = mean_epoch_loss.item()
        best_W = W
        best_b = b
    print(f"Epoch Loss: {torch.mean(torch.from_numpy(np.array(epoch_loss)))=}")

Epoch Loss: torch.mean(torch.from_numpy(np.array(epoch_loss)))=tensor(0.9937, dtype=torch.float64)
Epoch Loss: torch.mean(torch.from_numpy(np.array(epoch_loss)))=tensor(0.7549, dtype=torch.float64)
Epoch Loss: torch.mean(torch.from_numpy(np.array(epoch_loss)))=tensor(2.2464, dtype=torch.float64)
Epoch Loss: torch.mean(torch.from_numpy(np.array(epoch_loss)))=tensor(10.7577, dtype=torch.float64)
Epoch Loss: torch.mean(torch.from_numpy(np.array(epoch_loss)))=tensor(90.2122, dtype=torch.float64)


KeyboardInterrupt: 

## Model Testing

In [18]:
outputs = []

for test_data in test_loader:
    data, target = test_data["data"], test_data["target"]
    with torch.no_grad():
        resp = model(data, best_W, best_b)
    outputs.append(resp)


tensor([[1.4510, 1.0673, 1.2813,  ..., 2.2304, 2.2944, 2.4713],
        [0.9381, 2.0406, 2.3138,  ..., 2.1326, 2.4139, 1.9525],
        [2.4843, 2.8793, 1.4926,  ..., 2.4963, 1.4456, 2.4560],
        [2.1921, 2.8440, 2.3659,  ..., 1.2171, 2.5133, 2.6391]],
       device='cuda:0')

In [28]:
output_tensor = torch.cat(outputs, dim=-1).view(-1)

In [24]:
mse = lambda out1, out2: torch.mean((out1 - out2) ** 2)

In [30]:
output_tensor

tensor([1.4510, 1.0673, 1.2813,  ..., 1.2171, 2.5133, 2.6391], device='cuda:0')

In [31]:
y_test_tensor = torch.tensor(y_test, device=device)

In [32]:
mse(y_test_tensor, output_tensor).item()

18.372723926109867

## Training and Validation Loop

This section is dedicated to model training and validation in a structured manner. 

1. We are initializing the training loop for the model 
2. For every iteration, we are going to use multiple callbacks for easy training
3. After completion of model training, we are going to use the validation 

In [79]:
def loss_calc(model, data, target):
    model_output = model(data, W, b)
    y_output = target.view(-1) # Original Output
    model_output = model_output.view(-1) # Reshaping predicted output
    return torch.mean((y_output - model_output) ** 2)

def train_one_step(model, data, optimizer):
    """
    For each step, we have to follow
    - Initialize the optimizer gredients to zero
    - Perform forward Pass
    - Calculate batch error/loss
    - Take loss backward pass
    - Update weights using optimizer steps
    - Return loss 
    """
    loss = loss_calc(model, **data)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad() # This will re-initialize every gredients to zero
    return loss.item()

def train_one_epoch(model, data_loader, optimizer):
    """
    For each epoch, 
    - Iterate over the complete training data.
    - Calculate the step loss 
    - Compute the epoch average loss
    - Print the average loss and return to the user
    """
#     model.train() # Setting model state to train only works with module API
    total_loss = 0

    for idx, data in enumerate(data_loader):
        step_loss = train_one_step(model, data, optimizer)
        total_loss += step_loss
#         scheduler.step()
    return total_loss / (idx + 1)

In [80]:
# Validation Loop

def eval_one_epoch(model, data_loader):
#     model.eval() # Disabling the gredient calculation only works with Module API
    total_loss = 0
    for idx, data in enumerate(data_loader):
        with torch.no_grad():
            loss = loss_calc(model, **data)
            total_loss += loss.item()
    return total_loss / (idx + 1)

In [81]:
W = torch.randn((8, 1), dtype=torch.float32, device=device, requires_grad=True)
b = torch.randn(1, dtype=torch.float32, device=device, requires_grad=True)
def train_model(epochs):
    optimizer = torch.optim.Adam([W, b])
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)
    for epoch in tqdm(range(epochs)):
        train_loss = train_one_epoch(model, train_loader, optimizer)
        tqdm.write(f"Training Loss @ {epoch=}: {train_loss}")
        valid_loss = eval_one_epoch(model, test_loader)
        tqdm.write(f"Validation Loss @{epoch=}: {valid_loss}")

train_model(10)
    

  0%|          | 0/10 [00:00<?, ?it/s]

Training Loss @ epoch=0: 8.3791763422823
Validation Loss @epoch=0: 2.131917714143114
Training Loss @ epoch=1: 0.9859590346538263
Validation Loss @epoch=1: 0.7275682775892115
Training Loss @ epoch=2: 0.6317201307458671
Validation Loss @epoch=2: 0.6230825289935884
Training Loss @ epoch=3: 0.5571011994899271
Validation Loss @epoch=3: 0.6380147487971961
Training Loss @ epoch=4: 0.5363461275971017
Validation Loss @epoch=4: 0.666968245555145
Training Loss @ epoch=5: 0.5317289391160283
Validation Loss @epoch=5: 0.6966582339734395
Training Loss @ epoch=6: 0.5311922209253652
Validation Loss @epoch=6: 0.7230658169561462
Training Loss @ epoch=7: 0.5315681575440356
Validation Loss @epoch=7: 0.7458852321566479
Training Loss @ epoch=8: 0.5320888342636559
Validation Loss @epoch=8: 0.7655730084985379
Training Loss @ epoch=9: 0.5325827124998309
Validation Loss @epoch=9: 0.7826391488292016


# Linear Regression using Pytorch Module API


In [97]:
class RegressionModel(nn.Module):
    
    def __init__(self, inp_dim, out_dim):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(inp_dim, 512)
        self.fc2 = nn.Linear(512, out_dim)
        self.activation = nn.ReLU()
    
    def forward(self, data):
        x = self.fc1(data)
        x = self.fc2(x)
        return self.activation(x)

In [101]:
model = RegressionModel(8, 1).to(device)

In [113]:
def loss_calc(model, data, target):
    model_output = model(data)
    y_output = target.view(-1) # Original Output
    model_output = model_output.view(-1) # Reshaping predicted output
    return torch.mean((y_output - model_output) ** 2)

def train_one_step(model, data, optimizer):
    """
    For each step, we have to follow
    - Initialize the optimizer gredients to zero
    - Perform forward Pass
    - Calculate batch error/loss
    - Take loss backward pass
    - Update weights using optimizer steps
    - Return loss 
    """
    loss = loss_calc(model, **data)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad() # This will re-initialize every gredients to zero
    return loss.item()

def train_one_epoch(model, data_loader, optimizer):
    """
    For each epoch, 
    - Iterate over the complete training data.
    - Calculate the step loss 
    - Compute the epoch average loss
    - Print the average loss and return to the user
    """
    model.train() # Setting model state to train only works with module API
    total_loss = 0

    for idx, data in enumerate(data_loader):
        step_loss = train_one_step(model, data, optimizer)
        total_loss += step_loss
#         scheduler.step()
    return total_loss / (idx + 1)

In [114]:
# Validation Loop

def eval_one_epoch(model, data_loader):
    model.eval() # Disabling the gredient calculation only works with Module API
    total_loss = 0
    for idx, data in enumerate(data_loader):
        with torch.no_grad():
            loss = loss_calc(model, **data)
            total_loss += loss.item()
    return total_loss / (idx + 1)

In [115]:
def train_model(epochs):
    optimizer = torch.optim.Adam(model.parameters())
    for epoch in tqdm(range(epochs)):
        train_loss = train_one_epoch(model, train_loader, optimizer)
        tqdm.write(f"Training Loss @ {epoch=}: {train_loss}")
        valid_loss = eval_one_epoch(model, test_loader)
        tqdm.write(f"Validation Loss @{epoch=}: {valid_loss}")

train_model(10)
    

  0%|          | 0/10 [00:00<?, ?it/s]

Training Loss @ epoch=0: 0.6217589719412089
Validation Loss @epoch=0: 0.5757515416530236
Training Loss @ epoch=1: 0.5712738023626293
Validation Loss @epoch=1: 0.6074643328491699
Training Loss @ epoch=2: 0.5238559984895554
Validation Loss @epoch=2: 0.5782567869978794
Training Loss @ epoch=3: 0.5236196047128964
Validation Loss @epoch=3: 0.5347887690660955
Training Loss @ epoch=4: 0.5218709047054403
Validation Loss @epoch=4: 0.524489064716243
Training Loss @ epoch=5: 0.52273030110259
Validation Loss @epoch=5: 0.5151529439531859
Training Loss @ epoch=6: 0.52283830619002
Validation Loss @epoch=6: 0.5092880690074165
Training Loss @ epoch=7: 0.5229796854927576
Validation Loss @epoch=7: 0.5029654128331684
Training Loss @ epoch=8: 0.5873228463704351
Validation Loss @epoch=8: 0.5518924326772617
Training Loss @ epoch=9: 0.5120735686545823
Validation Loss @epoch=9: 0.5527669904097489


In [117]:
torch.save(model, "regression_model.")

<bound method Module.state_dict of RegressionModel(
  (fc1): Linear(in_features=8, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
  (activation): ReLU()
)>